In [1]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [2]:
import numpy as np
import pandas as pd
from utils.dataset import DataSet
from utils.generate_test_splits import generate_hold_out_split, read_ids

In [3]:
d = DataSet()
generate_hold_out_split(d, training=0.8)
trainID = set(read_ids("training_ids.txt", "splits"))
valID = set(read_ids("hold_out_ids.txt", "splits"))

Reading dataset
Total stances: 49972
Total bodies: 1683


# Hyperparams

In [4]:
MAX_SENT_PER_ART = 5
MAX_SENT_LEN = 20
MAX_VOCAB = 50000
VECTOR_SIZE = 300

# Set up training and validation data

In [5]:
train_stances = [stance for stance in d.stances if stance['Body ID'] in trainID]
train_headlines = [stance['Headline'] for stance in train_stances]
train_labels = [stance['Stance'] for stance in train_stances]
train_body = [d.articles[stance['Body ID']] for stance in train_stances]

val_stances = [stance for stance in d.stances if stance['Body ID'] in valID]
val_headlines = [stance['Headline'] for stance in val_stances]
val_labels = [stance['Stance'] for stance in val_stances]
val_body = [d.articles[stance['Body ID']] for stance in val_stances]

# Vectorization and Tokenization

In [6]:
import nltk
from nltk import tokenize
from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.layers import TextVectorization
nltk.download('punkt')

vectorizer = TextVectorization(max_tokens=MAX_VOCAB, output_sequence_length=MAX_SENT_LEN)
vectorizer.adapt(train_body + train_headlines + val_body + val_headlines)

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

[nltk_data] Downloading package punkt to /home/sw26wong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
sent_tok_art = []
for article in train_body:
    sent_tok_art.append(tokenize.sent_tokenize(article))

vsent_tok_art = []
for article in val_body:
    vsent_tok_art.append(tokenize.sent_tokenize(article))

In [8]:
X_train_body = np.zeros((len(train_stances), MAX_SENT_PER_ART, MAX_SENT_LEN), dtype='int32')

for i, article in enumerate(sent_tok_art):
    for j, sentence in enumerate(article[:MAX_SENT_PER_ART]):
        words = text_to_word_sequence(sentence)
        for k, word in enumerate(words[:MAX_SENT_LEN]):
            X_train_body[i][j][k] = word_index.get(word, 1) # get else UNK

X_train_head = np.zeros((len(train_stances), MAX_SENT_LEN), dtype='int32')

for i, headline in enumerate(train_headlines):
    words = text_to_word_sequence(headline)
    for j, word in enumerate(words[:MAX_SENT_LEN]):
        X_train_head[i][j] = word_index.get(word, 1)

X_val_body = np.zeros((len(val_stances), MAX_SENT_PER_ART, MAX_SENT_LEN), dtype='int32')

for i, article in enumerate(vsent_tok_art):
    for j, sentence in enumerate(article[:MAX_SENT_PER_ART]):
        words = text_to_word_sequence(sentence)
        for k, word in enumerate(words[:MAX_SENT_LEN]):
            X_val_body[i][j][k] = word_index.get(word, 1)

X_val_head = np.zeros((len(val_stances), MAX_SENT_LEN), dtype='int32')

for i, headline in enumerate(val_headlines):
    words = text_to_word_sequence(headline)
    for j, word in enumerate(words[:MAX_SENT_LEN]):
        X_val_head[i][j] = word_index.get(word, 1)

In [9]:
targets = pd.Series(train_labels)
one_hot = pd.get_dummies(targets,sparse = True)
one_hot_labels = np.asarray(one_hot)
y_train = one_hot_labels

In [10]:
targets = pd.Series(val_labels)
one_hot = pd.get_dummies(targets,sparse = True)
one_hot_labels = np.asarray(one_hot)
y_val = one_hot_labels

In [11]:
LABELS = ['agree', 'disagree', 'discuss', 'unrelated']

# Create Embedding Matrix from Google word2vec

In [12]:
vocab_size = len(word_index)
embedding_matrix = np.zeros((vocab_size+1, VECTOR_SIZE))

for word, i in word_index.items():
    try:
        v = wv[word]
        embedding_matrix[i] = v
    except KeyError:
        pass

In [13]:
import gc
del wv
gc.collect()

88

# Model

In [14]:
from keras.models import Sequential
from keras.layers import Dense,LSTM, TimeDistributed, Activation
from keras.layers import Flatten, Permute, merge, Input
from keras.layers import Embedding
from keras.models import Model
from keras.layers import Input,Dense,multiply,concatenate,Dropout
from keras.layers import GRU, Bidirectional

In [15]:
hidden_size = VECTOR_SIZE

sentence_input = Input(shape=(MAX_SENT_LEN,),dtype='int32')

embedded_sequences = Embedding(output_dim=hidden_size, 
                               input_dim=vocab_size+1, 
                               input_length=MAX_SENT_LEN,
                               weights=[embedding_matrix],
                               trainable=False,
                               mask_zero=True,
                    )(sentence_input)

l_LSTM = Bidirectional(LSTM(hidden_size,return_sequences = True))(embedded_sequences)
l_dense = TimeDistributed(Dense(hidden_size))(l_LSTM)
l_dense = Flatten()(l_dense)
sentEncoder = Model(sentence_input,l_dense)

body_input = Input(shape=(MAX_SENT_PER_ART,MAX_SENT_LEN,),dtype = 'int32')

body_encoder = TimeDistributed(sentEncoder)(body_input)

l_LSTM_sent = Bidirectional(LSTM(hidden_size,return_sequences=True))(body_encoder)
l_dense_sent = TimeDistributed(Dense(hidden_size))(l_LSTM_sent)
l_dense_sent = Flatten()(l_dense_sent)

heading_input = Input(shape = (MAX_SENT_LEN, ),dtype = 'int32')
heading_embedded_sequences = Embedding(output_dim=hidden_size, input_dim=vocab_size+1, \
                                       input_length = (MAX_SENT_LEN,), \
                                      weights = [embedding_matrix],
                                      trainable=False,
                                      mask_zero=True,)(heading_input)
h_dense = Dense(hidden_size,activation='relu')(heading_embedded_sequences)
h_flatten = Flatten()(h_dense)
article_output = concatenate([l_dense_sent,h_flatten],name = 'concatenate_heading')

news_vestor = Dense(hidden_size,activation = 'relu')(article_output)
preds = Dense(4,activation = 'softmax')(news_vestor)
model = Model([body_input,heading_input],[preds])

In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

In [17]:
model.fit([X_train_body,X_train_head],[y_train], validation_data=([X_val_body,X_val_head],[y_val]), epochs=15 , batch_size=128)

Epoch 1/15
316/316 [==============================] - 160s 475ms/step - loss: 0.5646 - categorical_accuracy: 0.7913 - val_loss: 0.7205 - val_categorical_accuracy: 0.7101
Epoch 2/15
316/316 [==============================] - 147s 465ms/step - loss: 0.2900 - categorical_accuracy: 0.8865 - val_loss: 0.7330 - val_categorical_accuracy: 0.7253
Epoch 3/15
316/316 [==============================] - 146s 463ms/step - loss: 0.1728 - categorical_accuracy: 0.9321 - val_loss: 0.6709 - val_categorical_accuracy: 0.7690
Epoch 4/15
316/316 [==============================] - 146s 463ms/step - loss: 0.1131 - categorical_accuracy: 0.9567 - val_loss: 0.7652 - val_categorical_accuracy: 0.7717
Epoch 5/15
316/316 [==============================] - 146s 462ms/step - loss: 0.0805 - categorical_accuracy: 0.9690 - val_loss: 0.7778 - val_categorical_accuracy: 0.7751
Epoch 6/15
316/316 [==============================] - 146s 461ms/step - loss: 0.0662 - categorical_accuracy: 0.9755 - val_loss: 0.8247 - val_categoric

In [18]:
cd = DataSet("competition_test")

test_stances = cd.stances
test_headlines = [stance['Headline'] for stance in test_stances]
test_labels = [stance['Stance'] for stance in test_stances]
test_body = [cd.articles[stance['Body ID']] for stance in test_stances]

X_test_body = np.zeros((len(test_stances), MAX_SENT_PER_ART, MAX_SENT_LEN), dtype='int32')
sent_tok_test = []
for article in test_body:
    sent_tok_test.append(tokenize.sent_tokenize(article))

for i, article in enumerate(sent_tok_test):
    for j, sentence in enumerate(article[:MAX_SENT_PER_ART]):
        words = text_to_word_sequence(sentence)
        for k, word in enumerate(words[:MAX_SENT_LEN]):
            X_test_body[i][j][k] = word_index.get(word, 1)

X_test_head = np.zeros((len(test_stances), MAX_SENT_LEN), dtype='int32')

for i, headline in enumerate(test_headlines):
    words = text_to_word_sequence(headline)
    for j, word in enumerate(words[:MAX_SENT_LEN]):
        X_test_head[i][j] = word_index.get(word, 1)


predictions = model.predict([X_test_body,X_test_head])
predicted_label = [LABELS[max([0, 1, 2, 3], key=lambda x: p[x])] for p in predictions]

Reading dataset
Total stances: 25413
Total bodies: 904


In [19]:
score = sum([pl == a for pl, a in zip(predicted_label, test_labels)])/len(test_labels)
score

0.7313186164561445

In [21]:
model.save("google300-embeddings-nontrainable")

INFO:tensorflow:Assets written to: google300-embeddings-nontrainable/assets


INFO:tensorflow:Assets written to: google300-embeddings-nontrainable/assets
